In [2]:

# ===============================================================
# 📥 NE PAS EXECUTER C'EST UN FICHIER QUI A DEJA été EXECUTER
# ===============================================================







import pandas as pd
import numpy as np

# ===============================================================
# 📥 Lecture du fichier source
# ===============================================================
nom_fichier = 'statistiques_communes_dvf_2020_2024.csv'
stats = pd.read_csv(nom_fichier, sep=';', low_memory=False)

# Nettoyage de base
stats = stats.dropna(subset=['nom_commune', 'annee'])
stats['annee'] = pd.to_numeric(stats['annee'], errors='coerce')
stats = stats.dropna(subset=['annee'])
stats['annee'] = stats['annee'].astype(int)

# ===============================================================
# 🏙️ Détection et fusion des arrondissements (Paris, Lyon, Marseille)
# ===============================================================
def ville_simplifiee(nom):
    nom = str(nom)
    if nom.lower().startswith("paris"):
        return "Paris"
    elif nom.lower().startswith("lyon"):
        return "Lyon"
    elif nom.lower().startswith("marseille"):
        return "Marseille"
    else:
        return nom

stats['ville'] = stats['nom_commune'].apply(ville_simplifiee)

# On détecte les colonnes numériques (pour les agréger)
colonnes_numeriques = stats.select_dtypes(include=[np.number]).columns.tolist()
colonnes_non_num = [c for c in stats.columns if c not in colonnes_numeriques]

# Agrégation par ville et année :
# - Pour les colonnes numériques → somme (ou moyenne pondérée si applicable)
# - Pour les colonnes non numériques → première valeur (logique pour une donnée descriptive)
def aggreg_flexible(df):
    data = {}
    for col in df.columns:
        if col in colonnes_numeriques:
            if col == 'prix_m2_moyen' and 'nb_transactions' in df.columns:
                data[col] = (df['prix_m2_moyen'] * df['nb_transactions']).sum() / df['nb_transactions'].sum()
            else:
                data[col] = df[col].sum()
        elif col == 'nom_commune':
            data[col] = df['ville'].iloc[0]  # remplace par "Paris", "Lyon" ou "Marseille"
        else:
            data[col] = df[col].dropna().iloc[0] if not df[col].dropna().empty else np.nan
    return pd.Series(data)

agg = (
    stats.groupby(['ville', 'annee'], dropna=False)
    .apply(aggreg_flexible)
    .reset_index(drop=True)
)

# ===============================================================
# 🗺️ Sélection des grandes villes françaises
# ===============================================================
villes_selection = [
    'Paris', 'Lyon', 'Marseille',
    'Toulouse', 'Bordeaux', 'Montpellier',
    'Lille', 'Rennes', 'Strasbourg', 'Grenoble',
]

# On garde uniquement les villes voulues (insensible à la casse)
stats_simplifie = agg[agg['ville'].str.lower().isin([v.lower() for v in villes_selection])].copy()

# ===============================================================
# 💾 Enregistrement du CSV simplifié
# ===============================================================
fichier_sortie = 'statistiques_grandes_villes_2020_2024_toutes_colonnes.csv'
stats_simplifie.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ Fichier simplifié enregistré : {fichier_sortie}")
print(f"Nombre de lignes : {len(stats_simplifie)}")
print(stats_simplifie[['ville', 'annee', 'prix_m2_moyen', 'nb_transactions']].head())

KeyboardInterrupt: 